### Añadimos SRC a la raíz del proyecto para poder importar el contenido

In [1]:
import sys
import os

# Obtener la ruta absoluta de la carpeta raíz (donde está src)
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Subir un nivel desde notebooks/

# Agregar la carpeta src al path
sys.path.append(os.path.join(ROOT_DIR, "src"))

### Importamos los módulos necesarios

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import json
import re
from collections import Counter
from datetime import datetime

from config import SMOGON_SCRAPPING_DATA_PATH

### Comenzamos la recolección de datos mediante Scraping

In [ ]:
# 📌 URL base de Smogon Showdown stats
BASE_URL = "https://www.smogon.com/stats/"

# 📌 Rango de años a analizar
YEARS = ["2022", "2023", "2024"]

# 📌 Lista de meses a analizar, incluyendo DLCs
MONTHS = [
    f"{year}-{str(month).zfill(2)}" for year in YEARS for month in range(1, 13)
] + ["2023-09-DLC1", "2023-12-DLC2"]  # Incluye meses con DLCs

# 📌 Diccionario con los archivos VGC relevantes
VGC_FILES = {
    "2022": ["gen8vgc2022-1760.txt"],  
    "2023": [
        "gen9vgc2023series2-1760.txt",
        "gen9vgc2023regulationc-1760.txt",
        "gen9vgc2023regulationd-1760.txt",
        "gen9vgc2023regulatione-1760.txt"
    ],  
    "2024": [
        "gen9vgc2024regg-1760.txt",
        "gen9vgc2024regh-1760.txt",
        "gen9vgc2024regf-1760.txt"
    ]
}

# 📌 Ajuste especial para los meses con DLC
DLC_FILES = {
    "2023-09-DLC1": ["gen9vgc2023regulationd-1760.txt"],
    "2023-12-DLC2": ["gen9vgc2023regulationf-1760.txt"]
}

# 📌 Diccionario para almacenar los datos de uso
vgc_usage = {year: {} for year in YEARS}

# 📌 Scraping de los archivos correspondientes
for month in MONTHS:
    year = month.split("-")[0]
    
    # Determinar qué archivos usar (DLC o estándar)
    files_to_scrape = DLC_FILES.get(month, VGC_FILES.get(year, []))

    for file_name in files_to_scrape:
        url = f"{BASE_URL}{month}/{file_name}"
        
        print(f"🔎 Accediendo a {url}")

        try:
            response = requests.get(url)
            if response.status_code == 200:
                lines = response.text.split("\n")

                # 📄 Mostrar las primeras líneas del archivo para verificar la estructura
                print(f"📄 Primeras líneas del archivo ({file_name} en {month}):")
                print("\n".join(lines[:10]))  # Muestra las primeras 10 líneas
                
                for line in lines:
                    # Evitar líneas vacías o encabezados
                    if not line.strip() or "Usage" in line:
                        continue
                    
                    # Intentar dividir la línea correctamente
                    parts = line.split("|")
                    
                    if len(parts) > 3:
                        try:
                            pokemon_name = parts[2].strip().lower().replace(" ", "-")  # Formato de nombres
                            usage_percent = parts[3].strip().replace("%", "")
                            
                            if pokemon_name and usage_percent:
                                usage_float = float(usage_percent)  # Convertir a float
                                vgc_usage[year].setdefault(pokemon_name, []).append(usage_float)
                        
                        except ValueError:
                            print(f"⚠️ Error al convertir datos en: {line}")

            else:
                print(f"⚠️ No se encontró data para {file_name} en {month}")

        except Exception as e:
            print(f"❌ Error al acceder a {url}: {e}")

    time.sleep(1)  # Evitar sobrecargar Smogon

# 📌 Obtener la moda (valor más frecuente) de cada Pokémon en cada año
final_usage = {}
for year, data in vgc_usage.items():
    final_usage[year] = {
        pokemon: Counter(usages).most_common(1)[0][0] if usages else "NoUsage"
        for pokemon, usages in data.items()
    }

# 📌 Convertir a DataFrame
df_vgc = pd.DataFrame(final_usage).reset_index()
df_vgc.rename(columns={"index": "Pokemon", "2022": "VGCusage2022", "2023": "VGCusage2023", "2024": "VGCusage2024"}, inplace=True)

# 📌 Guardar en CSV
df_vgc.to_csv(SMOGON_SCRAPPING_DATA_PATH, index=False)
print("✅ Datos de VGC guardados en {SMOGON_SCRAPPING_DATA_PATH}")



🔎 Accediendo a https://www.smogon.com/stats/2022-01/gen8vgc2022-1760.txt
📄 Primeras líneas del archivo (gen8vgc2022-1760.txt en 2022-01):
 Total battles: 818865
 Avg. weight/team: 0.002
 + ---- + ------------------ + --------- + ------ + ------- + ------ + ------- + 
 | Rank | Pokemon            | Usage %   | Raw    | %       | Real   | %       | 
 + ---- + ------------------ + --------- + ------ + ------- + ------ + ------- + 
 | 1    | Zacian-Crowned     | 60.39172% | 762697 | 46.570% | 520631 | 69.453% | 
 | 2    | Incineroar         | 58.57608% | 828744 | 50.603% | 366436 | 48.883% | 
 | 3    | Kyogre             | 49.39676% | 592063 | 36.151% | 377266 | 50.328% | 
 | 4    | Regieleki          | 33.66805% | 447449 | 27.321% | 154396 | 20.597% | 
 | 5    | Grimmsnarl         | 26.28657% | 370212 | 22.605% | 133457 | 17.803% | 
🔎 Accediendo a https://www.smogon.com/stats/2022-02/gen8vgc2022-1760.txt
📄 Primeras líneas del archivo (gen8vgc2022-1760.txt en 2022-02):
 Total battles: 7523

In [5]:
df_vgc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067 entries, 0 to 1066
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Pokemon       1067 non-null   object 
 1   VGCusage2022  725 non-null    float64
 2   VGCusage2023  624 non-null    float64
 3   VGCusage2024  794 non-null    float64
dtypes: float64(3), object(1)
memory usage: 33.5+ KB
